preprocess images 

In [1]:
import cv2
import numpy as np
from typing import List

def preprocess_images(images:List[np.ndarray]):
    gray_images = [cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) for image in images]
    resized_images = np.array([cv2.resize(gray_image, (32, 32)) for gray_image in gray_images])
    normalized_images = resized_images.astype('float32') / 255.0
    return np.expand_dims(normalized_images, axis=-1)

predict func

In [2]:
def predict_images(model, images:List[np.ndarray]):
    processed_images = preprocess_images(images)

    predictions = model.predict(processed_images)

    predicted_labels = [''.join(map(str, row)) for row in np.argmax(predictions, axis=2)]
    predicted_confs = np.min(np.max(predictions, axis=2),axis=1)

    return predicted_labels, predicted_confs

read png from dir

In [3]:
import glob
import os

def read_all_png(directory):
    pattern = os.path.join(directory, "*.png")
    png_files = glob.glob(pattern)
    images = [cv2.imread(file) for file in png_files]
    
    return images

plt show

In [4]:
import matplotlib.pyplot as plt
import random

def show_predict_result(model, directory):
    pattern = os.path.join(directory, "*.png")
    
    # 查找目录下所有的 PNG 文件
    png_files = glob.glob(pattern)

    print(png_files)
    
    # 使用 OpenCV 读取每个 PNG 文件
    images = [cv2.imread(file) for file in png_files]
    predicted_labels, predicted_confs = predict_images(model, images)

    plt.figure(figsize=(15, 7))
    for i in range(10):
        plt.subplot(2, 5, i + 1)
        plt.imshow(images[i], cmap='gray')
        plt.axis('off')
        plt.title(f'Predicted: {predicted_labels[i]}\nConf: {predicted_confs[i]:.2f}')
    plt.show()

In [5]:
import tensorflow as tf

model = tf.keras.models.load_model('models/svhn_2digits_model.keras')

show_predict_result(model, 'custom_images/')

2024-08-12 13:27:20.661814: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-12 13:27:20.663231: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-12 13:27:20.691269: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-12 13:27:20.691781: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-12 13:27:21.186757: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

OSError: No file or directory found at models/svhn_2digits_model.keras